
# Cardiovascular Stent Simulation


This example problem shows how to simulate stent-artery interaction during and after stent placement in an occluded artery.
The analysis exposes advanced modeling techniques using PyMAPDL such as:
* Contact
* Element birth and death
* Mixed u-P formulation
* Nonlinear stabilization

This example is inspired from the model and analysis defined in Chapter 25 of the Mechanical APDL Technology Showcase Manual.

## Additional Packages Used

* `Matplotlib <https://matplotlib.org>`_ is used for plotting purposes.<br>


## Setting up model

In [6]:
# starting MAPDL as a service and importing an external model
from ansys.mapdl.core import launch_mapdl

# start MAPDL as a service
mapdl = launch_mapdl()
print(mapdl)

Product:             Ansys Mechanical Enterprise
MAPDL Version:       22.2
ansys.mapdl Version: 0.63.3



### Defining material properties<br>

In [7]:
# define 316L Stainless steel
mapdl.prep7()
mapdl.mptemp()
mapdl.mptemp(sloc="1", t1="0")
mapdl.mpdata(lab="EX", mat="1", c1="200e3")
mapdl.mpdata(lab="PRXY", mat="1", c1="0.3")
mapdl.mptemp()
mapdl.mptemp(sloc="1", t1="0")
mapdl.mpdata(lab="DENS", mat="1", c1="8000e-9")

PROPERTY TABLE DENS  MAT=      1  NUM. POINTS=  1
 SLOC=   1    0.8000000E-05

### Defining element types

In [8]:
# for straight line segments
mapdl.et(itype="1", ename="beam189")
mapdl.sectype(secid="1", type_="beam", subtype="csolid")
mapdl.secdata(val1=0.05)

# for arcs
mapdl.et(itype="2", ename="beam189")
mapdl.sectype(secid="2", type_="beam", subtype="csolid")
mapdl.secdata(val1=0.05)

SECTION ID NUMBER IS:            2
   BEAM SECTION TYPE IS:     Circular Solid  
   BEAM SECTION NAME IS:             
   COMPUTED BEAM SECTION DATA SUMMARY:
    Area                 = 0.78479E-02
    Iyy                  = 0.48986E-05
    Iyz                  =-0.66174E-22
    Izz                  = 0.48986E-05
    Warping Constant     = 0.46449E-40
    Torsion Constant     = 0.97972E-05
    Centroid Y           =-0.26983E-18
    Centroid Z           =-0.48569E-18
    Shear Center Y       = 0.36554E-20
    Shear Center Z       = 0.18437E-17
    Shear Correction-xy  = 0.85691    
    Shear Correction-yz  = 0.35124E-15
    Shear Correction-xz  = 0.85691    
                 
    Beam Section is offset to CENTROID of cross section

### Defining 5-parameter Mooney-Rivlin hyperelastic artery material model

In [9]:
c10 = 18.90e-3
c01 = 2.75e-3
c20 = 590.43e-3
c11 = 857.2e-3
nu1 = 0.49
dd = 2 * (1 - 2 * nu1) / (c10 + c01)

In [10]:
mapdl.tb(lab="hyper", mat="2", npts="5", tbopt="mooney")
mapdl.tbdata(stloc="1", c1="c10", c2="c01", c3="c20", c4="c11", c6="dd")

DATA FOR  HYPE  TABLE FOR MATERIAL   2 AT TEMPERATURE=  0.0000    
 LOC=  1 7.88861e-31     7.88861e-31     7.88861e-31     7.88861e-31     0.00000e+00     7.88861e-31

### Defining linear elastic material model for stiff calcified plaque

In [11]:
mapdl.mp(lab="EX", mat="3", c0="00219e3")
mapdl.mp(lab="NUXY", mat="3", c0="0.49")

MATERIAL          3     NUXY =  0.4900000

### Define Solid185 element type to mesh both the artery and plaque

In [12]:
# for artery
mapdl.et(itype="9", ename="SOLID185")
mapdl.keyopt(
    itype="9", knum="6", value="1")  # Use mixed u-P formulation to avoid locking
mapdl.keyopt(itype="9", knum="2", value="3")  # Use Simplified Enhanced Strain method

# for plaque
mapdl.et(itype="16", ename="SOLID185")
mapdl.keyopt(itype="16", knum="2", value="0")  # Use B-bar

ELEMENT TYPE      16 IS SOLID185     3-D 8-NODE STRUCTURAL SOLID 
  KEYOPT( 1- 6)=        0      0      0        0      0      0
  KEYOPT( 7-12)=        0      0      0        0      0      0
  KEYOPT(13-18)=        0      0      0        0      0      0

 CURRENT NODAL DOF SET IS  UX    UY    UZ    ROTX  ROTY  ROTZ
  THREE-DIMENSIONAL MODEL

### Defining settings to model the stent, the artery and the plaque

Use force-distributed boundary constraints on 2 sides of artery wall to allow<br>
for radial expansion of tissue without rigid body motion.

#### Settings for MPC Surface-based, force-distributed contact on proximal plane parallel to x-y plane

In [13]:
mapdl.mat("2")
mapdl.r(nset="3")
mapdl.real(nset="3")
mapdl.et(itype="3", ename="170")
mapdl.et(itype="4", ename="174")
mapdl.keyopt(itype="4", knum="12", value="5")
mapdl.keyopt(itype="4", knum="4", value="1")
mapdl.keyopt(itype="4", knum="2", value="2")
mapdl.keyopt(itype="3", knum="2", value="1")
mapdl.keyopt(itype="3", knum="4", value="111111")
mapdl.type(itype="3")

mapdl.mat("2")
mapdl.r(nset="4")
mapdl.real(nset="4")
mapdl.et(itype="5", ename="170")
mapdl.et(itype="6", ename="174")
mapdl.keyopt(itype="6", knum="12", value="5")
mapdl.keyopt(itype="6", knum="4", value="1")
mapdl.keyopt(itype="6", knum="2", value="2")
mapdl.keyopt(itype="5", knum="2", value="1")
mapdl.keyopt(itype="5", knum="4", value="111111")
mapdl.type(itype="5")

ELEMENT TYPE SET TO         5

#### Settings for standard contact between stent and inner plaque wall contact surface

In [14]:
mapdl.mp(lab="MU", mat="1", c0="0")
mapdl.mat("1")
mapdl.mp(lab="EMIS", mat="1", c0="7.88860905221e-31")
mapdl.r(nset="6")
mapdl.real(nset="6")
mapdl.et(itype="10", ename="170")
mapdl.et(itype="11", ename="177")
mapdl.r(nset="6", r3="1.0", r4="1.0", r5="0")
mapdl.rmore(r9="1.0E20", r10="0.0", r11="1.0")
mapdl.rmore(r7="0.0", r8="0", r9="1.0", r10="0.05", r11="1.0", r12="0.5")
mapdl.rmore(r7="0", r8="1.0", r9="1.0", r10="0.0")
mapdl.keyopt(itype="11", knum="5", value="0")
mapdl.keyopt(itype="11", knum="7", value="1")
mapdl.keyopt(itype="11", knum="8", value="0")
mapdl.keyopt(itype="11", knum="9", value="0")
mapdl.keyopt(itype="11", knum="10", value="2")
mapdl.keyopt(itype="11", knum="11", value="0")
mapdl.keyopt(itype="11", knum="12", value="0")
mapdl.keyopt(itype="11", knum="2", value="3")
mapdl.keyopt(itype="10", knum="5", value="0")

ELEMENT TYPE      10 IS TARGE170     3-D TARGET SEGMENT          
  KEYOPT( 1- 6)=        0      0      0        0      0      0
  KEYOPT( 7-12)=        0      0      0        0      0      0
  KEYOPT(13-18)=        0      0      0        0      0      0

 CURRENT NODAL DOF SET IS  UX    UY    UZ    ROTX  ROTY  ROTZ
  THREE-DIMENSIONAL MODEL

#### Settings for MPC based, force-distributed constraint on proximal stent nodes

In [15]:
mapdl.mat("1")
mapdl.r(nset="7")
mapdl.real(nset="7")
mapdl.et(itype="12", ename="170")
mapdl.et(itype="13", ename="175")
mapdl.keyopt(itype="13", knum="12", value="5")
mapdl.keyopt(itype="13", knum="4", value="1")
mapdl.keyopt(itype="13", knum="2", value="2")
mapdl.keyopt(itype="12", knum="2", value="1")
mapdl.keyopt(itype="12", knum="4", value="111111")
mapdl.type(itype="12")

ELEMENT TYPE SET TO        12

#### Settings for MPC based, force-distributed constraint on distal stent nodes

In [16]:
mapdl.mat("1")
mapdl.r(nset="8")
mapdl.real(nset="8")
mapdl.et(itype="14", ename="170")
mapdl.et(itype="15", ename="175")
mapdl.keyopt(itype="15", knum="12", value="5")
mapdl.keyopt(itype="15", knum="4", value="1")
mapdl.keyopt(itype="15", knum="2", value="2")
mapdl.keyopt(itype="14", knum="2", value="1")
mapdl.keyopt(itype="14", knum="4", value="111111")
mapdl.type(itype="14")

ELEMENT TYPE SET TO        14

### Reading the geometry file

In [17]:
mapdl.cdread(option="db", fname="stent", ext="cdb")
mapdl.allsel(labt="all")
mapdl.finish()

***** ROUTINE COMPLETED *****  CP =         0.938

## Static Analysis

Run static analysis

In [18]:
# enter solution processor and define analysis settings
mapdl.run("/solu")
mapdl.antype(antype="0")
mapdl.nlgeom(key="on")

LARGE DEFORMATION ANALYSIS

### Apply Load Step 1: Balloon angioplasty of the artery to expand it past the radius of the stent - IGNORE STENT

In [19]:
mapdl.nsubst(nsbstp="20", nsbmx="20")
mapdl.nropt(option1="full")
mapdl.cncheck(option="auto")
mapdl.esel(type_="s", item="type", vmin="11")
mapdl.cm(cname="contact2", entity="elem")
mapdl.ekill(elem="contact2")  # Kill contact elements in stent-plaque contact 
                              #pair so that the stent is ignored in the first loadstep
mapdl.nsel(type_="s", item="loc", comp="x", vmin="0", vmax="0.01e-3")
mapdl.nsel(type_="r", item="loc", comp="y", vmin="0", vmax="0.01e-3")
mapdl.d(node="all", lab="all")
mapdl.allsel()

mapdl.sf(nlist="load", lab="pres", value="10e-2")  # Apply 0.1 Pa/mm^2 pressure to inner plaque wall
mapdl.allsel()
mapdl.nldiag(label="cont", key="iter")
mapdl.solve()
mapdl.save()

ALL CURRENT MAPDL DATA WRITTEN TO FILE NAME= file.db
  FOR POSSIBLE RESUME FROM THIS POINT

### Apply Load Step 2: Reactivate contact between stent and plaque

In [20]:
mapdl.ealive(elem="contact2")
mapdl.allsel()

mapdl.nsubst(nsbstp="2", nsbmx="2")
mapdl.save()
mapdl.solve()

*****  MAPDL SOLVE    COMMAND  *****

 *** NOTE ***                            CP =     100.125   TIME= 18:36:40
 Present time 0 is less than or equal to the previous time.  Time will   
 default to 2.                                                           

 *** MAPDL - ENGINEERING ANALYSIS SYSTEM  RELEASE 2022 R2          22.2     ***
 Ansys Mechanical Enterprise                       
 00000000  VERSION=WINDOWS x64   18:36:41  NOV 28, 2022 CP=    100.469

                                                                               



                      L O A D   S T E P   O P T I O N S

   LOAD STEP NUMBER. . . . . . . . . . . . . . . .     2
   TIME AT END OF THE LOAD STEP. . . . . . . . . .  2.0000    
   AUTOMATIC TIME STEPPING . . . . . . . . . . . .    ON
      INITIAL NUMBER OF SUBSTEPS . . . . . . . . .     2
      MAXIMUM NUMBER OF SUBSTEPS . . . . . . . . .     2
      MINIMUM NUMBER OF SUBSTEPS . . . . . . . . .     1
   MAXIMUM NUMBER OF EQUILIBRIUM ITERATIONS. .

### Apply Load Step 3

In [21]:
mapdl.nsubst(nsbstp="1", nsbmx="1", nsbmn="1")
mapdl.solve()

*****  MAPDL SOLVE    COMMAND  *****

 *** NOTE ***                            CP =     131.141   TIME= 18:37:11
 Present time 0 is less than or equal to the previous time.  Time will   
 default to 3.                                                           

 *** MAPDL - ENGINEERING ANALYSIS SYSTEM  RELEASE 2022 R2          22.2     ***
 Ansys Mechanical Enterprise                       
 00000000  VERSION=WINDOWS x64   18:37:11  NOV 28, 2022 CP=    131.391

                                                                               



                      L O A D   S T E P   O P T I O N S

   LOAD STEP NUMBER. . . . . . . . . . . . . . . .     3
   TIME AT END OF THE LOAD STEP. . . . . . . . . .  3.0000    
   AUTOMATIC TIME STEPPING . . . . . . . . . . . .    ON
      INITIAL NUMBER OF SUBSTEPS . . . . . . . . .     1
      MAXIMUM NUMBER OF SUBSTEPS . . . . . . . . .     1
      MINIMUM NUMBER OF SUBSTEPS . . . . . . . . .     1
   MAXIMUM NUMBER OF EQUILIBRIUM ITERATIONS. .

### Apply Load Step 4: Apply blood pressure (13.3 kPa) load to inner wall of plaque and allow the stent to act as a scaffold

In [22]:
mapdl.nsubst(nsbstp="300", nsbmx="3000", nsbmn="30")
mapdl.sf(nlist="load", lab="pres", value="13", value2="3e-3")
mapdl.allsel()

SELECT ALL ENTITIES OF TYPE= ALL  AND BELOW

### Apply stabilization with energy option

In [23]:
mapdl.stabilize(key="const", method="energy", value="0.1")

NONLINEAR STABILIZATION CONTROL:
 KEY=CONS   METHOD=ENER   VALUE= 0.10000       SUBSTPOPT=NO     FORCELIMIT= 0.20000

## Solve the model

In [24]:
mapdl.solve()
mapdl.save()
mapdl.finish()

# Work in process

## Post-processing the modal results

This sections illustrates different methods to post-process the results of the modal analysis : PyMAPDL method, PyMAPDL result reader, PyDPF-Post and PyDPF-Core. All methods lead to the same result and are just given as an example of how each module can be used.

In [ ]:
from ansys.dpf import core as dpf
from ansys.dpf.core import operators as ops
from ansys.dpf import post

In [ ]:
model = dpf.Model(mapdl.result_file)
ds = dpf.DataSources(mapdl.result_file)

In [ ]:
print(model)

In [ ]:
mesh = model.metadata.meshed_region
mesh.plot()

In [ ]:
u = model.results.displacement(time_scoping=[3]).eval() # mesh_scoping param 
print(u[0])
print(u.get_fields({"time":3})[0])

u[0].plot(deform_by = u[0])

In [ ]:
s_op = model.results.stress(time_scoping=[3])
s_op.inputs.requested_location.connect(dpf.locations.nodal)
s = s_op.eval()
s_VM = dpf.operators.invariant.von_mises_eqv_fc(fields_container=s).eval()



print(s)
print(s_VM)
# print(u[0])
# print(u.get_fields({"time":3})[0])

s_VM[0].plot(deform_by = u[0])

In [ ]:
op = dpf.operators.scoping.on_named_selection()

# Make input connections
op.inputs.requested_location.connect('nodal')
op.inputs.named_selection_name.connect('STENT')
op.inputs.int_inclusive.connect(1)
my_streams_container = dpf.StreamsContainer()
op.inputs.streams_container.connect(my_streams_container)
my_data_sources = dpf.DataSources()
op.inputs.data_sources.connect(my_data_sources)

In [ ]:
u = model.results.displacement(time_scoping=[3],mesh_scoping=model.nodes_scoping('STENT')).eval() # mesh_scoping param 
print(u[0])
print(u.get_fields({"time":3})[0])

u[0].plot(deform_by = u[0])

In [ ]:
stress_result = solution.stress()
stress = stress_result.tensor

In [ ]:
Pres = dpf.operators.result.raw_displacement()
Pres.inputs.data_sources.connect(ds)
out_pres = Pres.outputs.fields_container()

In [ ]:
result = dpf.operators.result.displacement()
result.inputs.data_sources.connect(ds)
freqids = list(range(1,4))
result.inputs.time_scoping.connect([1,2,3,4])
out = Pres.outputs.fields_container()

In [ ]:
RES = model.results.displacement.on_all_time_freqs.eval()

In [ ]:
Norm = dpf.operators.result.displacement_X()
Norm.inputs.data_sources.connect(ds)
Norm.inputs.time_scoping.connect(freqids)
ux = Norm.outputs.fields_container()

In [ ]:
scopingNS = dpf.operators.scoping.on_named_selection()
scopingNS.inputs.requested_location.connect("Nodal")
scopingNS.inputs.named_selection_name.connect("STENT")
scopingNS.inputs.int_inclusive.connect(0)
scopingNS.inputs.data_sources.connect(ds)

In [ ]:
scop = scopingNS.outputs.mesh_scoping()

In [ ]:
rescop = dpf.operators.scoping.rescope_fc()
rescop.inputs.fields_container(out)
rescop.inputs.mesh_scoping(scop)
out_node = rescop.outputs.fields_container()

In [ ]:
print(out_node[0].data)

In [ ]:
solution = post.load_solution(mapdl.result_file)

In [ ]:
disp = solution.displacement()
stress_result = solution.stress()
print(disp)
print('Hello')
print(stress_result)

In [ ]:
server_file_path_rst = dpf.upload_file_in_tmp_folder(oname+'.rst') #.rst file
model = dpf.Model(server_file_path_rst)
ds = dpf.DataSources(server_file_path_rst)

In [ ]:
stress = stress_result.tensor
print(stress)

In [ ]:
nodes_scoping = dpf.mesh_scoping_factory.named_selection_scoping("STENT", model)
disp_stent = model.results.displacement(mesh_scoping=nodes_scoping).eval()
model.metadata.meshed_region.plot(disp_stent)

In [ ]:
mapdl.list_files()

In [ ]:
mesh_scoping = model.metadata.named_selection("STENTB")
print(mesh_scoping)

In [ ]:
volume = model.results.elemental_volume(mesh_scoping=mesh_scoping).eval()
model.metadata.meshed_region.plot(volume)

In [ ]:
STOP

### Using DPF-Post

In [ ]:
from ansys.dpf import post

In [ ]:
solution_path = mapdl.result_file
solution = post.load_solution(solution_path)

In [ ]:
print(solution)

In [ ]:
for i in range(1,4): 
    print(i)
    displacement = solution.displacement(time_scoping=i)
    total_deformation = displacement.norm
    total_deformation.plot_contour(show_edges=True, background="w")

In [ ]:
displacement = solution.displacement(time_scoping=2)
total_deformation = displacement.norm
total_deformation.plot_contour(show_edges=True, background="w")

In [ ]:
solution_path = mapdl.result_file
solution = post.load_solution(solution_path)
print(solution)

In [ ]:
displacement = solution.displacement(time_scoping=2)
total_deformation = displacement.norm
total_deformation.plot_contour(show_edges=True, background="w")

In [ ]:
disp = displacement.vector
disp.num_fields

In [ ]:
disp.get_data_at_field(0)

In [ ]:
stress_result = solution.stress()
stress = stress_result.tensor

In [ ]:
stress.plot_contour()

### Using DPF-Core


In [ ]:
from ansys.dpf import core

In [ ]:
model = core.Model(solution_path)
results = model.results
print(results)
displacements = results.displacement()
total_def = core.operators.math.norm_fc(displacements)
total_def_container = total_def.outputs.fields_container()
mesh = model.metadata.meshed_region
mesh.plot(total_def_container.get_field_by_time_id(1))

#############################################################################<br>
Run PSD analysis<br>
----------------<br>
The response spectrum analysis is defined, solved and post-processed.

define PSD analysis with input spectrum

In [ ]:
mapdl.slashsolu()
mapdl.antype("spectr")

power spectral density

In [ ]:
mapdl.spopt("psd")

use input table 1 with acceleration spectrum in terms of acceleration due to<br>
gravity

In [ ]:
mapdl.psdunit(1, "accg", 9.81 * 1000)

define the frequency points in the input table 1

In [ ]:
mapdl.psdfrq(1, "", 1, 40, 50, 70.71678, 100, 700, 900)

define the PSD values in the input table 1

In [ ]:
mapdl.psdval(1, 0.01, 0.01, 0.1, 1, 10, 10, 1)

set the damping ratio as 5%

In [ ]:
mapdl.dmprat(0.05)

apply base excitation on the set of nodes N_BASE_EXCITE in the y-direction<br>
from table 1

In [ ]:
mapdl.d("N_BASE_EXCITE", "uy", 1)

calculate the participation factor for PSD with base excitation from input<br>
table 1

In [ ]:
mapdl.pfact(1, "base")

write the displacent solution relative to the base excitation to the results<br>
file from the PSD analysis

In [ ]:
mapdl.psdres("disp", "rel")

write the absolute velocity solution to the results file from the PSD analysis

In [ ]:
mapdl.psdres("velo", "abs")

write the absolute acceleration solution to the results file from the PSD<br>
analysis

In [ ]:
mapdl.psdres("acel", "abs")

combine only those modes whose significance level exceeds 0.0001

In [ ]:
mapdl.psdcom()
output = mapdl.solve()
print(output)

#############################################################################<br>
Post-process PSD analysis<br>
~~~~~~~~~~~~~~~~~~~~~~~~~<br>
The response spectrum analysis is post-processed. First, the standard<br>
MAPDL POST1 postprocessor is used. Then, the MAPDL time-history<br>
POST26 postprocessor is used to generate the response power spectral<br>
density.<br>
<br>
.. note::<br>
   The graph generated through POST26 is exported as a picture in the working<br>
   directory. Finally, the results from POST26 are saved to Python variables<br>
   to be plotted in the Python environment with the use of Matplotlib<br>
   library.

#############################################################################<br>
Post-process PSD analysis in POST1<br>
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

In [ ]:
mapdl.post1()
mapdl.set(1, 1)
mapdl.plnsol("u", "sum")
mapdl.set("last")
mapdl.plnsol("u", "sum")

#############################################################################<br>
Post-process PSD analysis in POST26 (time-history post-processing)<br>
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

In [ ]:
mapdl.post26()

allow storage for 200 variables

In [ ]:
mapdl.numvar(200)
mapdl.cmsel("s", "MY_MONITOR")
monitored_node = mapdl.queries.ndnext(0)
mapdl.store("psd")

store the psd analysis u_y data for the node MYMONITOR as the reference no 2

In [ ]:
mapdl.nsol(2, monitored_node, "u", "y")

compute the response power spectral density for displacement associated with<br>
variable 2

In [ ]:
mapdl.rpsd(3, 2)
mapdl.show("png")

plot the variable 3

In [ ]:
mapdl.plvar(3)

print the variable 3

In [ ]:
mapdl.prvar(3)

x-axis is set for Log X scale

In [ ]:
mapdl.gropt("logx", 1)

y-axis is set for Log X scale

In [ ]:
mapdl.gropt("logy", 1)

plot the variable 3

In [ ]:
mapdl.plvar(3)
mapdl.show("close")

#############################################################################<br>
Post-process PSD analysis using Matplotlib<br>
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

store MAPDL results to python variables

In [ ]:
mapdl.dim("frequencies", "array", 4000, 1)
mapdl.dim("response", "array", 4000, 1)
mapdl.vget("frequencies", 1)
mapdl.vget("response", 3)
frequencies = mapdl.parameters["frequencies"]
response = mapdl.parameters["response"]

use Matplotlib to create graph

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111)
plt.xscale("log")
plt.yscale("log")
ax.plot(frequencies, response)
ax.set_xlabel("Frequencies")
ax.set_ylabel("Response power spectral density")

## Exit MAPDL

In [ ]:
mapdl.exit()